# Workbook for automated creation  of CMWP files for a pre-integrated Zr XRD pattern

This workbook takes a folder of .dat files and creates a .bg-spline.dat and .peak-index.dat file based on Zr peak positions.

## Import stuff

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt
import glob
import csv
from shutil import copyfile

from src.xrd_tools import getReflections
from src.cmwp_tools import getBaseline, getPeaks

## Define settings

In [2]:
base = '/home/rhys/Documents/CMWP-200623/testdata/'         # This is the base directory where the data is stored
template = base + '0template/'                              # This is where the template files are stored. These are copied for each .dat file.

wavelength = 1.5406                                         # Wavelength in angstrom

peak_name, peak_pos = getReflections(crystalType='hcp', 
            a=3.232, c=5.147, wavelength=wavelength, printReflections=False)

baseline=[30.1, 42, 52, 61, 72, 85, 101.5, 114.8]           # What 2theta points to calculte the background spline from
baseline_interpolate = None

searchrange = 20                                            # How many data points each side of the approximate 2theta peak position to search for the true peak

## Find files

In [3]:
files = glob.glob(base + '*.dat')
template_files = glob.glob(template + '*')

files = [x for x in files if 'bg-spline' not in x]
files = [x for x in files if 'peak-index' not in x]
    
print('Found {0} files'.format(len(files)))
for file in files: print('\t{0}'.format(file))
print('\nFound {0} template files:'.format(len(template_files)))
for file in template_files: print('\t{0}'.format(file))

Found 1 files
	/home/rhys/Documents/CMWP-200623/testdata/W7-3_24hr-summed.dat

Found 4 template files:
	/home/rhys/Documents/CMWP-200623/testdata/0template/template.dat.q.ini
	/home/rhys/Documents/CMWP-200623/testdata/0template/template.dat.fit_params.ini
	/home/rhys/Documents/CMWP-200623/testdata/0template/template.dat.ini
	/home/rhys/Documents/CMWP-200623/testdata/0template/template.dat.fit.ini


## Run

In [8]:
for i, file in enumerate(files):
    
    print('\rCurrent:   {0} / {1}'.format(i+1, len(files)), end='')
    
    for t_file in template_files:
         copyfile(t_file, file.split('.')[-2] + t_file.split('/')[-1][8:])
        
    ########################### Load data ###########################
    
    data = np.loadtxt(file)
    xvals = data[:,0]
    yvals = data[:,1]

    ########################### Make figure ###########################

    plt.ioff()
    fig, (ax) = plt.subplots(1, 1, figsize=(16,8))
    ax.set_title(file.split('.')[-2].split('/')[-1]);
    ax.set_xlabel('2theta (deg)', fontsize=14); 
    ax.set_ylabel('Intensity', fontsize=14);
    plt.subplots_adjust(left=0.1, bottom=0.1, top=0.9, right=0.9, wspace=0.115, hspace=0)
    plt.subplots_adjust(hspace=0)
    
    ax.plot(xvals, yvals)
    ax.set_xlim(np.min(xvals)+0.01, np.max(xvals)-0.01)
    ax.set_ylim(np.min(yvals)-5, np.max(yvals)*1.1)

    ########################### Make bg-spline ###########################
    
    baseline_pos, baseline_int, cs = getBaseline(xvals, yvals, 
                                                 baseline, baseline_interpolate, 
                                                 file.split('.')[-2]+'.bg-spline.dat')
    
    ########################### Plot spline ###########################

    ax.plot(xvals, cs(xvals))
    ax.plot(baseline_pos, baseline_int, 'o',c='r')
        
    ########################### Make peak-index ###########################

    peak_pos, peak_name, peak_int = getPeaks(xvals, yvals, 
                                             peak_pos, peak_name, searchrange,
                                             file.split('.')[-2]+'.peak-index.dat')

    ########################### Plot peaks ###########################
    
    ax.vlines(peak_pos, ymin=np.min(yvals), ymax=np.max(yvals), alpha=0.1, colors='r')
    
    for pos, intensity, name in zip(peak_pos, peak_int+10, peak_name):
        ax.text(pos, intensity, name, horizontalalignment = 'center', c='r')

    plt.yscale('log')
    plt.savefig(file.split('.')[-2]+'_log.png', dpi=200)
    plt.close()
print('\rDone                 '.format(i, len(files)), end='')

Done                 